<a href="https://colab.research.google.com/github/ido177/colab/blob/main/join/join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.33MB [00:00, 93.2MB/s]


# Загрузка БД

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
sqlite3.sqlite_version

'3.36.0'

In [3]:
con = sqlite3.connect('db')

In [4]:
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 27.3MB/s]


In [5]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [6]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


head выводит 5 первых значений, tail - 5 последних


In [7]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

to_datetime меняет формат данных в дату
% помогает указать переменную

In [8]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

возможные типы данных

In [9]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

Заливаем таблицу в БД
Название, выбор бд, наличие индекса, замена существующих

In [10]:
def select(sql):
  return pd.read_sql(sql,con)

функция селекта

# JOIN

In [ ]:
users = pd.DataFrame({'id': [1,2,3],
                      'name': ['john', 'gleb', 'tyrion']})

In [ ]:
users

,id,name
0,1,john
1,2,gleb
2,3,tyrion


In [ ]:
items = pd.DataFrame({'user_id': [1,3],
                      'item_name': ['hleb', 'gold'],
                      'value': [5, 100]})

In [ ]:
items

,user_id,item_name,value
0,1,hleb,5
1,3,gold,100


In [ ]:
users.to_sql('users', con, index=False, if_exists='replace')
items.to_sql('items', con, index=False, if_exists='replace')

In [ ]:
sql = '''
select * from users t
left join items i 
on t.id = i.user_id
'''

In [ ]:
select(sql)

,id,name,user_id,item_name,value
0,1,john,1.0,hleb,5.0
1,2,gleb,NaN,None,NaN
2,3,tyrion,3.0,gold,100.0


Left join - все слева и то, что нашлось, справа
where i.item_name is not null - имитирует inner

In [ ]:
sql = '''
select * from users t
inner join items i 
on t.id = i.user_id
'''

In [ ]:
select(sql)

,id,name,user_id,item_name,value
0,1,john,1,hleb,5
1,3,tyrion,3,gold,100


inner join - только при полных совпадениях, без NaN
нет совпадения по id - скипаем строку

In [ ]:
users = pd.DataFrame({'id': [1,2,3],
                      'name': ['john', 'gleb', 'tyrion'],
                      'victory': [2,10,1]
                      })

добавляем повтор по id

In [ ]:
sql = '''
select * from users t
left join items i 
on t.id = i.user_id
'''

In [ ]:
sql = '''
select * from users t
inner join items i 
on t.id = i.user_id
'''

In [ ]:
select(sql)

,id,name,victory,user_id,item_name,value
0,1,john,2,1.0,hleb,5.0
1,2,gleb,10,NaN,None,NaN
2,3,tyrion,1,3.0,gold,100.0


In [ ]:
t_left = select(sql)

In [ ]:
t_left['victory'].sum()

13

In [ ]:
select(sql)

,id,name,victory,user_id,item_name,value
0,1,john,2,1,hleb,5
1,3,tyrion,1,3,gold,100


In [ ]:
t_right = select(sql)

In [ ]:
t_right['victory'].sum()

3

In [ ]:
sql = '''
select t.id, t.name, sum(t.victory) as victory_sum, count(i.item_name) as item_cnt, sum(i.value) as value_s

from users t
left join items i 
on t.id = i.user_id

group by t.id, t.name
'''

In [ ]:
select(sql)

,id,name,victory_sum,item_cnt,value_s
0,1,john,2,1,5.0
1,2,gleb,10,0,NaN
2,3,tyrion,1,1,100.0


In [ ]:
sql = '''
select t.id, t.name, t.victory as victory_sum, count(i.item_name) as item_cnt, sum(i.value) as value_s

from users t
left join items i 
on t.id = i.user_id

group by t.id, t.name, t.victory
'''

In [ ]:
select(sql)

,id,name,victory_sum,item_cnt,value_s
0,1,john,2,1,5.0
1,2,gleb,10,0,NaN
2,3,tyrion,1,1,100.0


In [ ]:
sql = ''' 
with items_agg as (
select t.user_id, count(t.item_name) as item_cnt, sum(value) as value_sum 
from items t
group by t.user_id
)

select *
from users t

left join items_agg i 
on t.id = i.user_id'''

In [ ]:
select(sql)

,id,name,victory,user_id,item_cnt,value_sum
0,1,john,2,1.0,1.0,5.0
1,2,gleb,10,NaN,NaN,NaN
2,3,tyrion,1,3.0,1.0,100.0


In [ ]:
sql = '''
select * from users t
FULL join items i 
on t.id = i.user_id
'''

FULL JOIN - возвращает все

In [ ]:
sql = '''
select * from users t
left join items i 
on t.id = i.user_id
union
select * from items i
left join users t
on i.user_id = t.id
'''

эквивалент full join, но с left

In [ ]:
select(sql)

,id,name,victory,user_id,item_name,value
0,1,hleb,5,1.0,john,2.0
1,1,john,2,1.0,hleb,5.0
2,2,gleb,10,NaN,None,NaN
3,3,gold,100,3.0,tyrion,1.0
4,3,tyrion,1,3.0,gold,100.0


In [ ]:
sql = ''' select * from date
join month on 1 = 1

cross join 1 = 1. Селектит бещз привязки к колонкам

In [15]:
t = pd.DataFrame({'dt':pd.to_datetime(['2021-04-01','2021-04-02','2021-04-03'], format='%Y-%m-%d'),
                                       'revenue': [1,2,3]})

In [16]:
t

,dt,revenue
0,2021-04-01,1
1,2021-04-02,2
2,2021-04-03,3


In [19]:
t.to_sql('revenue', con, index=False, if_exists='replace')

In [24]:
sql = ''' 
select * from revenue t
join revenue y 
on t.dt >= y.dt
'''

In [25]:
select(sql)

,dt,revenue,dt,revenue
0,2021-04-01 00:00:00,1,2021-04-01 00:00:00,1
1,2021-04-02 00:00:00,2,2021-04-01 00:00:00,1
2,2021-04-02 00:00:00,2,2021-04-02 00:00:00,2
3,2021-04-03 00:00:00,3,2021-04-01 00:00:00,1
4,2021-04-03 00:00:00,3,2021-04-02 00:00:00,2
5,2021-04-03 00:00:00,3,2021-04-03 00:00:00,3


In [30]:
sql = ''' 
select t.dt, t.revenue, sum(y.revenue) as cum_sum from revenue t
join revenue y 
on t.dt >= y.dt
group by t.dt, t.revenue
'''

делаем нарастающий итог

In [31]:
select(sql)

,dt,revenue,cum_sum
0,2021-04-01 00:00:00,1,1
1,2021-04-02 00:00:00,2,3
2,2021-04-03 00:00:00,3,6
